In [1]:
# Import dependencies
import json
import pandas as pd
import numpy as np
import sklearn as skl
import numpy
import datetime
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# From dependency imports
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score,r2_score
from sklearn.linear_model import LogisticRegression
from sqlalchemy import create_engine
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorf

In [2]:
### Configure settings for Relational Database Service, and defining database info in config variable.
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {'user': 'postgres', 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver",
          "location": "34.67.52.115",
          "db": "team5k",
          "port": "5432"}

In [3]:
### Flag to print Informational message
dmdebug=True

### Creating variable holding postgres info in format "postgres://[user]:[password]@[location]:[port]/[database]".
create_engine_str = ('postgresql://' 
                     + config["user"] 
                     + ":" + config["password"] 
                     + "@" + config["location"] 
                     + ":" + config["port"] 
                     + "/" + config["db"])

In [4]:
# Connecting to Postgres instance.
engine = create_engine(create_engine_str)

In [5]:
# Printing info for table names. 
print (engine.table_names())

['committees', 'candidates', 'education', 'committee_summary_2020', 'six_state_donations', 'donations', 'health_metrics', 'agg_county_votes', 'birth_death_rate', 'postal_codes', 'fec_committee', 'agg_county_donors', 'pres_votes_6t', 'unemployment']


In [6]:
### Get donor data
donors_df = pd.read_sql_query('select * from "agg_county_donors"',con=engine)
donors_df

,index,blue_amt,red_amt,other_amt,total_amt,blue_num,red_num,other_num,total_num,county,state,election_year
0,0,0,95080,0,95080,0,193,0,193,Mohave,AZ,2000
1,0,3557230,21404116,0,24961346,4743,36491,0,41234,Maricopa,AZ,2000
2,0,1969873,2990403,0,4960276,2627,5646,0,8273,Pima,AZ,2000
3,0,34850,495230,0,530080,69,740,0,809,Yavapai,AZ,2000
4,0,800,46400,0,47200,4,93,0,97,Cochise,AZ,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
6743,0,0,250,0,250,0,1,0,1,Chippewa\n taylor,WI,2020
6744,0,250,2800,0,3050,1,1,0,2,Burnett\n douglas\n washburn,WI,2020
6745,0,0,250,0,250,0,1,0,1,Columbia\n dane\n sauk,WI,2020
6746,0,0,220,0,220,0,1,0,1,Pepin\n pierce,WI,2020


In [7]:
# Generate our categorical variable list
donors_cat = donors_df.dtypes[donors_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
donors_df[donors_cat].nunique()

county    1238
state        6
dtype: int64

In [8]:
# creating instance of labelencoder
le = LabelEncoder()
# Assigning numerical values and storing in another column

donor_encoded_df = donors_df.apply(le.fit_transform)
donor_encoded_df

,index,blue_amt,red_amt,other_amt,total_amt,blue_num,red_num,other_num,total_num,county,state,election_year
0,0,0,2040,0,2250,0,193,0,193,1005,0,0
1,0,3394,4344,0,5067,1008,1443,0,1783,956,0,0
2,0,3307,4120,0,4875,912,1290,0,1624,1102,0,0
3,0,1215,3399,0,3922,69,649,0,738,1235,0,0
4,0,71,1391,0,1501,4,93,0,97,424,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6743,0,0,13,0,15,0,1,0,1,368,5,5
6744,0,28,141,0,149,1,1,0,2,265,5,5
6745,0,0,13,0,15,0,1,0,1,441,5,5
6746,0,0,10,0,11,0,1,0,1,1093,5,5


In [9]:
# Fit and transform the OneHotEncoder using the categorical variable list
#encode_df = pd.DataFrame(enc.fit_transform(nn_df[votes_mi_cat]))

# Add the encoded variable names to the DataFrame
#encode_df.columns = enc.get_feature_names(votes_mi_cat)
#encode_df.head()
X = donor_encoded_df
X.columns.to_list()
X = X.fillna(0)
X.isnull().sum().values.any()

False

In [10]:
# Create a y series from Transactions column
y = X["total_amt"]

In [11]:
# Split X and y into training and testing sets
X = X.drop("total_amt", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

In [14]:
# What is the length of the X_train info
len(X_train_scaled[0])

11

In [15]:
# First hidden layer
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
    )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [17]:
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [18]:
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 96        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 147
Trainable params: 147
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model using the "adam" optimizer and "mean_squared_error" or "mean_absolute_error" loss function
nn.compile(loss="mean_absolute_error", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model for 100 epochs.
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
5061/5061 [==============================] - 1s 153us/sample - loss: 2062.1301 - acc: 0.0000e+00
Epoch 2/100
5061/5061 [==============================] - 0s 82us/sample - loss: 2061.9023 - acc: 0.0000e+00
Epoch 3/100
5061/5061 [==============================] - 0s 83us/sample - loss: 2061.8538 - acc: 0.0000e+00
Epoch 4/100
5061/5061 [==============================] - ETA: 0s - loss: 2059.3993 - acc: 0.0000e+ - 0s 88us/sample - loss: 2061.8416 - acc: 0.0000e+00
Epoch 5/100
5061/5061 [==============================] - 1s 171us/sample - loss: 2061.8372 - acc: 0.0000e+00
Epoch 6/100
5061/5061 [==============================] - 1s 203us/sample - loss: 2061.8352 - acc: 0.0000e+00
Epoch 7/100
5061/5061 [==============================] - 1s 129us/sample - loss: 2061.8343 - acc: 0.0000e+00
Epoch 8/100
5061/5061 [==============================] - 1s 146us/sample - loss: 2061.8337 - acc: 0.0000e+00
Epoch 9/100
5061/5061 [==============================] - 0s 92us/sample - loss: 2061.83

5061/5061 [==============================] - 1s 152us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 76/100
5061/5061 [==============================] - 1s 137us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 77/100
5061/5061 [==============================] - 1s 107us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 78/100
5061/5061 [==============================] - 1s 107us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 79/100
5061/5061 [==============================] - 0s 90us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 80/100
5061/5061 [==============================] - 0s 93us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 81/100
5061/5061 [==============================] - 0s 88us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 82/100
5061/5061 [==============================] - 0s 88us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 83/100
5061/5061 [==============================] - 1s 102us/sample - loss: 2061.8326 - acc: 0.0000e+00
Epoch 84/100
5061/5061 [===

In [21]:
# Show model info summary
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1687/1687 - 0s - loss: 2081.6141 - acc: 5.9277e-04
Loss: 2081.614107449661, Accuracy: 0.0005927682504989207


In [22]:
# Predict values for the train and test sets
y_pred = nn.predict(X_test_scaled)
y_pred_train = nn.predict(X_train_scaled)
y_pred_train

array([[1.       ],
       [0.9999999],
       [1.       ],
       ...,
       [1.       ],
       [1.       ],
       [1.       ]], dtype=float32)

In [23]:
# Score the training predictions with r2_score()
r2_score(y_train, y_pred_train)

-1.68807435073651